# 01_LangChain Setup with Local Ollama Container Instance

This notebook demonstrates how to set up LangChain to work with a local Ollama container instance.
We will cover the installation of necessary packages, configuration, and connection to the local Ollama instance.
This could be useful for using LangChain/LangFlow like techniques.

## Step 1: Install Necessary Packages

First, we need to install the required packages. Run the following command to install LangChain dependencies.

Typically, this is already done with a `pip install -r requirements.txt`, and is only needed once.

In [1]:
%pip install -q langchain langchain_community langchain_ollama

Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Packages

Next, we will import the necessary packages for our setup.

In [2]:
import requests

## Step 3: Configure Connection to Local Ollama Instance

We need to configure the connection to our local Ollama container instance. The following code sets up the connection.

### Step 3.1: Setup Ollama Docker Container Instance

*_Run the following step first. This step is only needed if the following step fails._*

#### Ollama
Ollama is a containerized environment for running and managing LLMs. It provides an API for interacting with the models.

#### Setup Instructions
1. Ensure Docker is installed and running on your machine.
2. Check if there is a Docker container instance 'ollama' that can be (re-)started.
3. _If no 'ollama' container exists_: Create and run a new Ollama container instance using the following command:
   ```
   docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
   ```
4. Verify the Ollama instance is running by accessing [http://localhost:11434/api/version](http://localhost:11434/api/version).
5. Verify programmatic connection by (re-)running the following cell:


In [3]:
OLLAMA_API_URL = "http://localhost:11434/api"

def get_ollama_version():
    response = requests.get(f"{OLLAMA_API_URL}/version")
    if response.status_code == 200:
        return response.json()
    else:
        return None

try:
    ollama_version = get_ollama_version()
    if ollama_version:
        print(f"Connected to Ollama version: {ollama_version}")
    else:
        print("Failed to connect to Ollama instance.")
except requests.exceptions.ConnectionError:
    print("Failed to connect to Ollama instance, is the Docker container running?")
    input("Press Enter to continue...")
    raise

Connected to Ollama version: {'version': '0.3.13'}


## Step 4: Minimal Documentation and Instructions

### LangChain
LangChain is a framework for building applications with large language models (LLMs). It provides tools and abstractions to simplify the development process.

https://python.langchain.com/docs/tutorials/


## Step 5: Demonstrate LangChain

In this step, we will demonstrate basic LangChain usage.

### Example: Text Generation

Use LangChain to generate text based on a prompt.



In [4]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="llama3.2")
generated_text = model.invoke("Come up with 10 names for a song about parrots")

generated_text

'Here are ten potential song title ideas about parrots:\n\n1. "Rainbow Wings"\n2. "Parrot\'s Lament"\n3. "Squawk of the Wild"\n4. "Blue and Green Eyes"\n5. "Flock to the Sky"\n6. "Love in Full Plumage"\n7. "The Parrot\'s Songbook"\n8. "Colorful Dreams"\n9. "A Bird with a View"\n10. "Sunset Scales"\n\nThese titles aim to capture the vibrant colors, playful personalities, and captivating songs of parrots, while also exploring themes of freedom, beauty, and connection.'

### References

- [LangChain Documentation](https://python.langchain.com/docs/)

### Furture Work Suggestion

- [LangChain and DSpy Integration](https://www.reddit.com/r/LangChain/comments/1cqexk6/thoughts_on_dspy/)

## Step 6: User Input for Software Project Idea Using LangChain

In this step, we will prompt the user to write a software project idea, send it to the LLM, and display the feedback, summary, and plan.

### Example: User Input and Feedback Loop

1. Prompt the user to write a software project idea.
2. Send the idea to the LLM and display the feedback, summary, and plan.
3. Implement a refinement loop to allow the user to provide additional input and receive updated feedback.

#### Prompt User for Software Project Idea
We will prompt the user to write a software project idea.

In [5]:
# 7.1 Get user input for software project idea, default to 
project_idea = input("Please write your software project idea: ") or "Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents."
print(project_idea)

Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents.


In [6]:
prompt_template = """A user came up with a project idea: {project_idea}

The user wants to know what the project would look like if it was implemented.

Write a short description of the project, including the main features and how it would work.

Then give your feedback on the project idea, including any suggestions for improvement, or
areas that may need clarification.
"""

In [7]:
filled_prompt = prompt_template.format(project_idea=project_idea)
print(filled_prompt)

A user came up with a project idea: Create an AI software factory that generates software from project ideas, using step-by-step software processes that are implemented by LLM Agents.

The user wants to know what the project would look like if it was implemented.

Write a short description of the project, including the main features and how it would work.

Then give your feedback on the project idea, including any suggestions for improvement, or
areas that may need clarification.



In [8]:
project_idea_feedback = model.invoke(filled_prompt)
print(project_idea_feedback)


Project Description:

Project Name: AI Software Factory (ASF)

The AI Software Factory is an innovative platform that utilizes Large Language Model (LLM) Agents to generate software from project ideas. The ASF will consist of a step-by-step software development process implemented by the LLM Agents, which will guide users through the creation of their desired software product.

Main Features:

1. Project Idea Input: Users will input their project idea into the system, and the LLM Agent will analyze it to identify key features, requirements, and potential technologies.
2. Software Process Planning: The LLM Agent will create a customized software development process plan based on the user's input, outlining tasks, milestones, and deadlines.
3. Automated Code Generation: Using natural language processing (NLP) and machine learning algorithms, the LLM Agent will generate code for each task in the process plan, following industry-standard programming languages and best practices.
4. Integra

### Step 7. Implement Refinement Loop

We will implement a refinement loop to allow the user to provide additional input and receive updated feedback.

In [9]:
while True:
    additional_input = input(f"""
                             Please provide additional input to refine your project idea (or type 'exit' to finish):
                             
                             {project_idea}
                             """)
    if not additional_input.strip():
        break

    refinement_prompt_template = """
    You are an AI assistant. The user has provided a project description and additional input to refine the project idea.
    Provide an updated project description that addresses the user's input.

    {project_idea}

    Additional Input: {additional_input}
    """
    
    refinement_prompt = refinement_prompt_template.format(project_idea=project_idea, additional_input=additional_input)
    updated_feedback_summary_plan = model.invoke(refinement_prompt)
    print(updated_feedback_summary_plan)